# Experiment 007: Greedy Backtracking with Beam Search

Implement the blueshyy-style greedy backtracking approach:
- For each N, try removing different trees and see which removal minimizes the bounding box
- Use beam search with depth and lookahead to find optimal removal sequences
- This can propagate good configurations from larger N to smaller N

**Starting point:** 84.712432
**Target:** Improve by 1-3 points

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import os
import shutil
import time
import random

getcontext().prec = 25
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
# ChristmasTree class
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w, trunk_h = Decimal('0.15'), Decimal('0.2')
        base_w, mid_w, top_w = Decimal('0.7'), Decimal('0.4'), Decimal('0.25')
        tip_y, tier_1_y, tier_2_y, base_y = Decimal('0.8'), Decimal('0.5'), Decimal('0.25'), Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))
        # Cache bounds for fast access
        b = self.polygon.bounds
        self.bounds = (b[0]/float(scale_factor), b[1]/float(scale_factor), b[2]/float(scale_factor), b[3]/float(scale_factor))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
# Fast scoring and helper functions
def get_bounds_side(bounds_list):
    """Get side length from list of (minx, miny, maxx, maxy) bounds."""
    if not bounds_list:
        return 0.0
    minx = min(b[0] for b in bounds_list)
    miny = min(b[1] for b in bounds_list)
    maxx = max(b[2] for b in bounds_list)
    maxy = max(b[3] for b in bounds_list)
    return max(maxx - minx, maxy - miny)

def compute_touching_candidates(bounds_list):
    """Find indices of trees touching the bounding box boundary."""
    if not bounds_list:
        return []
    minx = min(b[0] for b in bounds_list)
    miny = min(b[1] for b in bounds_list)
    maxx = max(b[2] for b in bounds_list)
    maxy = max(b[3] for b in bounds_list)
    
    eps = 1e-9
    candidates = []
    for i, b in enumerate(bounds_list):
        if (abs(b[0] - minx) < eps or abs(b[2] - maxx) < eps or
            abs(b[1] - miny) < eps or abs(b[3] - maxy) < eps):
            candidates.append(i)
    return candidates if candidates else list(range(len(bounds_list)))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        for idx in tree_index.query(poly):
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

print("Helper functions defined")

Helper functions defined


In [4]:
# Greedy backtracking with beam search
def choose_removal_beam_lookahead(bounds_list, depth=3, beam=5, max_states=1000, seed=42):
    """Find best tree to remove using beam search."""
    rng = random.Random(seed)
    n0 = len(bounds_list)
    if n0 <= 1:
        return None, 0.0
    
    # Get candidates touching the boundary
    base_cands = compute_touching_candidates(bounds_list)
    
    # First layer: try removing each candidate
    first_layer = []
    for idx in base_cands:
        reduced = bounds_list[:idx] + bounds_list[idx+1:]
        s1 = get_bounds_side(reduced)
        first_layer.append((s1, reduced, idx, s1))
    
    if not first_layer:
        return None, float('inf')
    
    first_layer.sort(key=lambda x: x[0])
    frontier = first_layer[:min(beam, len(first_layer))]
    
    best_key = (frontier[0][0], frontier[0][3])
    best_first = frontier[0][2]
    best_s1 = frontier[0][3]
    
    states_used = len(frontier)
    
    # Expand to depth
    for d in range(2, max(2, depth + 1)):
        new_frontier = []
        for score_now, bds, first_idx, first_s1 in frontier:
            if len(bds) <= 1:
                key = (0.0, first_s1)
                if key < best_key:
                    best_key, best_first, best_s1 = key, first_idx, first_s1
                continue
            
            cands = compute_touching_candidates(bds)
            for j in cands:
                nb = bds[:j] + bds[j+1:]
                s = get_bounds_side(nb)
                new_frontier.append((s, nb, first_idx, first_s1))
                states_used += 1
                if states_used >= max_states:
                    break
            if states_used >= max_states:
                break
        
        if not new_frontier:
            break
        
        new_frontier.sort(key=lambda x: x[0])
        frontier = new_frontier[:min(beam, len(new_frontier))]
        
        cur_best = frontier[0]
        key = (cur_best[0], cur_best[3])
        if key < best_key:
            best_key, best_first, best_s1 = key, cur_best[2], cur_best[3]
        
        if states_used >= max_states:
            break
    
    return best_first, best_s1

print("Beam search function defined")

Beam search function defined


In [5]:
# Load current best submission
df = pd.read_csv('/home/code/experiments/005_extended_optimization/submission_fd.csv')

# Parse into tree lists
def strip_s(val):
    s = str(val)
    return s[1:] if s.startswith('s') else s

dict_of_tree_list = {}
dict_of_side_length = {}

for n in range(1, 201):
    gid = f"{n:03d}"
    group = df[df['id'].str.startswith(f'{gid}_')]
    if len(group) == n:
        trees = []
        for _, row in group.iterrows():
            trees.append(ChristmasTree(
                center_x=strip_s(row['x']),
                center_y=strip_s(row['y']),
                angle=strip_s(row['deg'])
            ))
        dict_of_tree_list[gid] = trees
        dict_of_side_length[gid] = get_bounds_side([t.bounds for t in trees])

initial_score = sum(dict_of_side_length[f"{n:03d}"]**2 / n for n in range(1, 201))
print(f"Initial score: {initial_score:.6f}")

Initial score: 84.712432


In [6]:
# Run greedy backtracking
print("\n" + "=" * 60)
print("GREEDY BACKTRACKING WITH BEAM SEARCH")
print("=" * 60)

DEPTH = 5
BEAM = 10
MAX_STATES = 2000
PASSES = 3
EPS_IMPROVE = 1e-9

total_improvements = 0

for pass_id in range(1, PASSES + 1):
    print(f"\n--- Pass {pass_id}/{PASSES} ---")
    improvements_this_pass = 0
    
    # Process from N=200 down to N=2
    for N in range(200, 1, -1):
        gidN = f"{N:03d}"
        gidPrev = f"{N-1:03d}"
        
        if gidN not in dict_of_tree_list or gidPrev not in dict_of_side_length:
            continue
        
        trees_N = dict_of_tree_list[gidN]
        bounds_list = [t.bounds for t in trees_N]
        prev_best = dict_of_side_length[gidPrev]
        
        # Find best tree to remove
        seed = 42 + pass_id * 1000 + N
        best_idx, best_s1 = choose_removal_beam_lookahead(
            bounds_list, depth=DEPTH, beam=BEAM, max_states=MAX_STATES, seed=seed
        )
        
        if best_idx is None:
            continue
        
        # Check if this improves N-1
        if best_s1 < prev_best - EPS_IMPROVE:
            # Create new tree list by removing best_idx
            new_trees = trees_N[:best_idx] + trees_N[best_idx+1:]
            
            # Verify no overlaps
            if not has_overlap(new_trees):
                old_side = dict_of_side_length[gidPrev]
                dict_of_tree_list[gidPrev] = new_trees
                dict_of_side_length[gidPrev] = best_s1
                improvements_this_pass += 1
                total_improvements += 1
                
                if improvements_this_pass <= 10:  # Print first 10
                    print(f"  N={N-1}: {old_side:.6f} -> {best_s1:.6f} (from N={N})")
    
    print(f"  Pass {pass_id} improvements: {improvements_this_pass}")
    if improvements_this_pass == 0:
        print("  No improvements - stopping early")
        break

print(f"\nTotal improvements: {total_improvements}")


GREEDY BACKTRACKING WITH BEAM SEARCH

--- Pass 1/3 ---


  Pass 1 improvements: 0
  No improvements - stopping early

Total improvements: 0


In [7]:
# Calculate new score
new_score = sum(dict_of_side_length[f"{n:03d}"]**2 / n for n in range(1, 201))
print(f"\nNew score: {new_score:.6f}")
print(f"Improvement: {initial_score - new_score:.6f}")


New score: 84.712432
Improvement: 0.000000


In [8]:
# Save the result
print("\nSaving result...")
rows = []
for n in range(1, 201):
    gid = f"{n:03d}"
    if gid in dict_of_tree_list:
        for i, tree in enumerate(dict_of_tree_list[gid]):
            rows.append({
                'id': f'{gid}_{i}',
                'x': f's{float(tree.center_x)}',
                'y': f's{float(tree.center_y)}',
                'deg': f's{float(tree.angle)}'
            })

result_df = pd.DataFrame(rows)
result_df.to_csv('/home/code/experiments/007_greedy_backtracking/submission_gb.csv', index=False)
print("Saved to submission_gb.csv")


Saving result...
Saved to submission_gb.csv


In [9]:
# Validate for overlaps
print("\nValidating...")
overlaps = []
for n in range(1, 201):
    gid = f"{n:03d}"
    if gid in dict_of_tree_list:
        if has_overlap(dict_of_tree_list[gid]):
            overlaps.append(n)

print(f"Overlapping groups: {overlaps}")
print(f"Validation: {'PASSED' if len(overlaps) == 0 else 'FAILED'}")

if len(overlaps) == 0:
    shutil.copy('/home/code/experiments/007_greedy_backtracking/submission_gb.csv',
                '/home/submission/submission.csv')
    print("Copied to /home/submission/submission.csv")

print(f"\n=== SUMMARY ===")
print(f"Starting: {initial_score:.6f}")
print(f"Final: {new_score:.6f}")
print(f"Improvement: {initial_score - new_score:.6f}")
print(f"Target: 68.931058")
print(f"Gap to target: {new_score - 68.931058:.6f}")


Validating...


Overlapping groups: []
Validation: PASSED
Copied to /home/submission/submission.csv

=== SUMMARY ===
Starting: 84.712432
Final: 84.712432
Improvement: 0.000000
Target: 68.931058
Gap to target: 15.781374
